In [2]:
import glob
import os
import numpy as np
from PIL import Image
from tqdm.auto import tqdm

from monai.transforms import Resize, EnsureChannelFirst, LoadImage, Compose, ScaleIntensity
from monai.data import ArrayDataset
from torch.utils.data import DataLoader


In [3]:
image_size = [512, 512]

transformer = Compose([LoadImage(image_only=True),
                       EnsureChannelFirst(),
                       ScaleIntensity(),
                       Resize(image_size)])

train_image_path = "data/REFUGE2/Train/Images/"
train_dm_path = "data/REFUGE2/Train/Disc_Masks/"
test_image_path = "data/REFUGE2/Test/Images/"
test_dm_path = "data/REFUGE2/Test/Disc_Masks/"
val_image_path = "data/REFUGE2/Validation/Images/"
val_dm_path = "data/REFUGE2/Validation/Disc_Masks/"

train_data = ArrayDataset(img=sorted([train_image_path + file for file in os.listdir(train_image_path)]),
                          img_transform=transformer,
                          seg=sorted([train_dm_path + file for file in os.listdir(train_dm_path)]),
                          seg_transform=transformer)

train_dataloader = DataLoader(train_data,
                              batch_size=1,
                              shuffle=False,
                              num_workers=2)

val_data = ArrayDataset(img=sorted([val_image_path + file for file in os.listdir(val_image_path)]),
                        img_transform=transformer,
                        seg=sorted([val_dm_path + file for file in os.listdir(val_dm_path)]),
                        seg_transform=transformer)

val_dataloader = DataLoader(val_data,
                            batch_size=1,
                            shuffle=False,
                            num_workers=2)

test_data = ArrayDataset(img=sorted([test_image_path + file for file in os.listdir(test_image_path)]),
                         img_transform=transformer,
                         seg=sorted([test_dm_path + file for file in os.listdir(test_dm_path)]),
                         seg_transform=transformer)

test_dataloader = DataLoader(test_data,
                             batch_size=1,
                             shuffle=False,
                             num_workers=2)

In [9]:
new_path = "data_test/REFUGE2/Train/"

if not os.path.exists(new_path + "Disc_Masks"):
    os.makedirs(new_path + "Disc_Masks")
    
if not os.path.exists(new_path + "Images"):
    os.makedirs(new_path + "Images")

for j, batch in tqdm(enumerate(train_dataloader)):
    new_image = Image.fromarray(np.array(batch[0][0].permute(1, 2, 0) * 255).astype(np.uint8))
    new_image.save(new_path + "Images/" + str(j) + ".png")
    new_image = Image.fromarray(np.array(batch[1][0].permute(1, 2, 0) * 255).astype(np.uint8))
    new_image.save(new_path + "Disc_Masks/" + str(j) + ".png")

0it [00:00, ?it/s]